### Face Detection and Tracking

In [4]:
# Importing needed packages
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os

In [6]:
# Getting name of haar cascade files
files = os.listdir(cv2.data.haarcascades)

for file in files:
    print(file)

haarcascade_upperbody.xml
haarcascade_frontalcatface_extended.xml
haarcascade_profileface.xml
haarcascade_frontalcatface.xml
haarcascade_frontalface_alt2.xml
haarcascade_eye.xml
__init__.py
haarcascade_lefteye_2splits.xml
__pycache__
haarcascade_frontalface_alt_tree.xml
haarcascade_righteye_2splits.xml
haarcascade_frontalface_alt.xml
haarcascade_lowerbody.xml
haarcascade_russian_plate_number.xml
haarcascade_frontalface_default.xml
haarcascade_smile.xml
haarcascade_license_plate_rus_16stages.xml
haarcascade_fullbody.xml
haarcascade_eye_tree_eyeglasses.xml
